In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns   

In [2]:
df = pd.read_csv("Dataset.csv")

In [3]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,VIDEO_NAME,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",PSY - GANGNAM STYLE(?????) M/V,1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,PSY - GANGNAM STYLE(?????) M/V,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,PSY - GANGNAM STYLE(?????) M/V,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,PSY - GANGNAM STYLE(?????) M/V,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,PSY - GANGNAM STYLE(?????) M/V,1


In [4]:
df.drop(columns=['DATE','AUTHOR', 'VIDEO_NAME'], inplace=True)

In [5]:
df.head()

,COMMENT_ID,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [6]:
df.describe()

,CLASS
count,1956.000000
mean,0.513804
std,0.499937
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
df['CLASS'].value_counts()

CLASS
1    1005
0     951
Name: count, dtype: int64

## text cleaning


In [8]:
import re

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
   # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    return text


In [9]:
preprocess_text("😫😓😏😪😔😖😌😭😎😚😘😙😗😋😝😜😛😍😒😞every single types of face on earth﻿")

'every single types of face on earth'

In [10]:
df['CONTENT'] = df['CONTENT'].apply(preprocess_text)

In [11]:
df['CONTENT'].head()

0    huh anyway check out this youtube channel koby...
1    hey guys check out my new channel and our firs...
2                just for test i have to say murdevcom
3           me shaking my sexy ass on my channel enjoy
4                     watchvvtarggvgtwq check this out
Name: CONTENT, dtype: object

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, precision_score

In [13]:
X=df['CONTENT']
y=df['CLASS']

In [14]:
X

0       huh anyway check out this youtube channel koby...
1       hey guys check out my new channel and our firs...
2                   just for test i have to say murdevcom
3              me shaking my sexy ass on my channel enjoy
4                        watchvvtarggvgtwq check this out
                              ...                        
1951    i love this song because we sing it at camp al...
1952    i love this song for two reasons 1it is about ...
1953                                                  wow
1954                              shakira u are so wiredo
1955                           shakira is the best dancer
Name: CONTENT, Length: 1956, dtype: object

In [15]:
y

0       1
1       1
2       1
3       1
4       1
       ..
1951    0
1952    0
1953    0
1954    0
1955    0
Name: CLASS, Length: 1956, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

**Training the spam-detection model**

In [17]:
spam_pipeline = make_pipeline(
    TfidfVectorizer(stop_words='english'),
    MultinomialNB()
)

In [18]:
spam_pipeline.fit(X_train,y_train)

,steps,"[('tfidfvectorizer', ...), ('multinomialnb', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


**Evaluating the model**

In [19]:
y_pred = spam_pipeline.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(classification_report(y_test, y_pred)) 
print(precision_score(y_test, y_pred))


0.875
              precision    recall  f1-score   support

           0       0.90      0.83      0.87       191
           1       0.85      0.92      0.88       201

    accuracy                           0.88       392
   macro avg       0.88      0.87      0.87       392
weighted avg       0.88      0.88      0.87       392

0.8518518518518519


## PREDICTION TIME


In [20]:
print(spam_pipeline.predict([preprocess_text("Congratulations! You've won a free ticket to Bahamas. Click here to claim now!")]))

[1]


**Saving the trained model**

In [21]:
import pickle

In [22]:
model_filename = 'spam_model.pkl' 
with open(model_filename, 'wb') as file:
    pickle.dump(spam_pipeline, file)

